# Start

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
purchase_data = pd.read_excel("./Analyst_dataset.xlsx", sheet_name='Purchase Exit Survey Data')
airings_data = pd.read_excel("./Analyst_dataset.xlsx", sheet_name='Airings')
lookup_data = pd.read_excel("./Analyst_dataset.xlsx", sheet_name='Lookup', skiprows=1)

# Preprocessing

In [3]:
# Lookup data is meant to facilitate a join between the purchases data and the spend/lift data, but lookup data has a row will all null values, which doesn't help the join in any way.  I'll drop it.
lookup_data = lookup_data.dropna(how='all')

In [4]:
# In order to make sure the joins happen correctly, we need to ensure that the strings we're joining on actually match.  

# The purchase data table is pretty messy, but if we assume the second column always contains the names of the networks, we can use .iloc to grab them and ensure they're lowercase 

lookup_data['Exit Survey'] = lookup_data['Exit Survey'].str.lower()
lookup_data['Airings'] = lookup_data['Airings'].str.upper()
airings_data['Network'] = airings_data['Network'].str.upper()
purchase_data.iloc[:, 1] = purchase_data.iloc[:, 1].str.lower()

# Transposing Purchase Exit Survey Data - Converting dates from columns to rows in Purchase Exit Survey Data

The purchases table is extremely messy and would be easier to work with if the rows were dates and the columns were the networks.  If we assume that the first row will always have the year, the third row will always have month names, and the fourth row will always have the day numbers, I can programmatically concatenate all the necessary date information in the form "Year-Month-Day", then use them for the rows.

In [5]:
current_year = purchase_data.iloc[0,:].dropna()
current_year = int(current_year)
current_year

2017

In [6]:
months = []
for month in purchase_data.iloc[2,2:].dropna():
    months.append(month)
months

['September', 'October']

In [7]:
# Grab the row of day numbers and cast as integers
day_nums = np.array(purchase_data.iloc[3,2:], dtype=int)

parsed_dates = []
current_month = months[0]
i = 0

# Walk through the list of day_nums.  

# If current_day_num > next_day_num, that indicates a change in month (ex: If current_day = Sept-30 and next_day = Oct-1, b/c 30 > 1).  When this happens, we concatenate the current_day, then increase i by 1 to set the current_month to the next month for furture concatenation.  

# If current_day_num < next_day_num, that indicates both days are in the same month (ex: If current_day = Sept-5 and next_day = Sept-6, b/c 5 < 6), so we concatenate like normal.

# The try block handles the exception when you get to the last day in day_nums.  Since there are no more days in the list, we get an error when we try to index into the list one day into the future.
for count, current_day_num in enumerate(day_nums, start=1):
    try:
        next_day_num = day_nums[count]
    except:
        pass
    if current_day_num > next_day_num:
        current_date = str(current_year) + '-' + current_month + '-' + str(current_day_num)
        current_date = datetime.strptime(current_date, '%Y-%B-%d').date()
        i += 1
        current_month = months[i]
        parsed_dates.append(current_date)
    else:
        current_date = str(current_year) + '-' + current_month + '-' + str(current_day_num)
        current_date = datetime.strptime(current_date, '%Y-%B-%d').date()
        parsed_dates.append(current_date)

# Now that all the dates have been parsed, we replace the unparsed dates with the parsed ones, then transpose the table.  We now have rows that correspond to dates and columns that correspond to networks
purchase_data.iloc[3,2:] = parsed_dates
purchase_data_transpose = purchase_data.iloc[3:,:].transpose()

## Some Cleanup

In [8]:
# Set the column of dates as the index and rename the axis appropriately
#purchase_data_transpose.index = purchase_data_transpose.iloc[:, 0]
purchase_data_transpose.set_index(3, inplace=True)
purchase_data_transpose.rename_axis('date', inplace=True)

# Drop first row, which doesn't contain anything useful
purchase_data_transpose = purchase_data_transpose.iloc[1:]

# Replace column names with the row of network names and then drop that row
purchase_data_transpose.columns = purchase_data_transpose.iloc[0]
purchase_data_transpose = purchase_data_transpose.drop(labels='source')

# Rename column axis as upper-case "Source" to match original table
purchase_data_transpose.rename_axis('Source', axis='columns', inplace=True)

# Convert index of dates to datetime objects
purchase_data_transpose.index = pd.to_datetime(purchase_data_transpose.index)

In [9]:
purchase_data_transpose.shape

(56, 30)

In [10]:
purchase_data_transpose.head()

Source,(blank),aapka_colors,baby_first,bloomberg,cbs_sports,cnbc,cnn,comedy_central,dateline,dish_network,...,other,science,sony_entertainment_tv,spectrum_sports,star_plus,television_food_network,the_history_channel,turner_network_tv,willow_tv,zeetv
date,,,,,,,,,,,,,,,,,,,,,
2017-09-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
2017-09-03,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2017-09-04,NaN,NaN,NaN,NaN,NaN,1.0,3.0,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
2017-09-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2017-09-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,2.0,1.0,NaN,NaN,NaN,1.0,NaN


## Done

# Metrics by Network

## Sum of Purchases by Network

In [11]:
purchases_by_network = purchase_data_transpose.sum(axis=0)
purchases_by_network = purchases_by_network.to_frame()
purchases_by_network = purchases_by_network.rename(columns={0:'Purchases'})

In [12]:
purchases_by_network.shape

(30, 1)

In [13]:
purchases_by_network.head()

,Purchases
Source,
(blank),3.0
aapka_colors,6.0
baby_first,1.0
bloomberg,2.0
cbs_sports,1.0


## Joining Purchases to Lookup Data

In [14]:
lookup_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 0 to 43
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Exit Survey    43 non-null     object
 1   Airings        38 non-null     object
 2   Exit Survey.1  43 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [15]:
purchases_by_network_w_lookup = lookup_data.merge(right=purchases_by_network, left_on='Exit Survey', right_on='Source', how='left')
purchases_by_network_w_lookup.drop(labels='Exit Survey.1', axis=1, inplace=True)
#purchases_by_network_w_lookup.rename(columns={0:'Purchases'}, inplace=True)
purchases_by_network_w_lookup.set_index('Exit Survey', inplace=True)
purchases_by_network_w_lookup.shape

(43, 2)

In [16]:
purchases_by_network_w_lookup.head()

,Airings,Purchases
Exit Survey,,
(blank),NaN,3.0
aapka_colors,NaN,6.0
baby_first,BABY,1.0
bloomberg,BLOM,2.0
cbs_sports,CBSS,1.0


## Spend and Lift by Network

In [17]:
spend_and_lift_by_network = airings_data.groupby('Network')[['Spend', 'Lift']].agg('sum')
spend_and_lift_by_network.shape

(19, 2)

In [18]:
spend_and_lift_by_network.head()

,Spend,Lift
Network,,
BLOM,4966.72,199
CNBC,6481.42,406
CNBCWORLD,1299.99,51
CNN,18114.35,1081
COM,7501.25,108


## Joining Purchases/Lookup to Spend and Lift

In [20]:
purchases_spend_lift_by_network = purchases_by_network_w_lookup.merge(right=spend_and_lift_by_network, left_on='Airings', right_index=True, how='left')
purchases_spend_lift_by_network.shape

(43, 4)

In [21]:
purchases_spend_lift_by_network.head()

,Airings,Purchases,Spend,Lift
Exit Survey,,,,
(blank),NaN,3.0,NaN,NaN
aapka_colors,NaN,6.0,NaN,NaN
baby_first,BABY,1.0,NaN,NaN
bloomberg,BLOM,2.0,4966.72,199.0
cbs_sports,CBSS,1.0,NaN,NaN


## Computing Metrics by Network

In [22]:
purchases_spend_lift_by_network['Conversion Rate'] = purchases_spend_lift_by_network['Purchases'] / purchases_spend_lift_by_network['Lift'] * 100

purchases_spend_lift_by_network['Cost Per Acquisition'] = purchases_spend_lift_by_network['Spend'] / purchases_spend_lift_by_network['Purchases']

purchases_spend_lift_by_network['Cost Per Visitor'] = purchases_spend_lift_by_network['Spend'] / purchases_spend_lift_by_network['Lift']

purchases_spend_lift_by_network['Percent of Purchases'] = purchases_spend_lift_by_network['Purchases'] / sum(purchases_spend_lift_by_network['Purchases'].fillna(0)) * 100

purchases_spend_lift_by_network['Percent of Spend'] = purchases_spend_lift_by_network['Spend'] / sum(purchases_spend_lift_by_network['Spend'].fillna(0)) * 100

purchases_spend_lift_by_network['Percent Pur > Percent Spend'] = purchases_spend_lift_by_network['Percent of Purchases'] > purchases_spend_lift_by_network['Percent of Spend']
git 
purchases_spend_lift_by_network

,Airings,Purchases,Spend,Lift,Conversion Rate,Cost Per Acquisition,Cost Per Visitor,Percent of Purchases,Percent of Spend,Percent Pur > Percent Spend
Exit Survey,,,,,,,,,,
(blank),NaN,3.0,NaN,NaN,NaN,NaN,NaN,1.271186,NaN,False
aapka_colors,NaN,6.0,NaN,NaN,NaN,NaN,NaN,2.542373,NaN,False
baby_first,BABY,1.0,NaN,NaN,NaN,NaN,NaN,0.423729,NaN,False
bloomberg,BLOM,2.0,4966.72,199.0,1.005025,2483.36,24.958392,0.847458,2.242951,False
cbs_sports,CBSS,1.0,NaN,NaN,NaN,NaN,NaN,0.423729,NaN,False
cnbc,CNBC,15.0,6481.42,406.0,3.694581,432.094667,15.964089,6.355932,2.926984,True
cnn,CNN,23.0,18114.35,1081.0,2.12766,787.580435,16.757031,9.745763,8.180369,True
comedy_central,COM,2.0,7501.25,108.0,1.851852,3750.625,69.456019,0.847458,3.387535,False
cozi,COZI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [23]:
purchases_spend_lift_by_network.shape

(43, 10)

## Done

# Grouped Metrics by Network and Month

## Purchase Data by Network and Month

In [ ]:
purchase_data_by_month = purchase_data_transpose.groupby(pd.Grouper(freq='M')).agg('sum')
purchase_data_by_month

In [ ]:
purchase_data_by_month = purchase_data_by_month.transpose()
purchase_data_by_month = purchase_data_by_month.stack().to_frame()
purchase_data_by_month.rename(columns={0:'Purchases'}, inplace=True)
purchase_data_by_month = purchase_data_by_month.reset_index()
purchase_data_by_month['Source'] = purchase_data_by_month['Source'].str.lower()
purchase_data_by_month

In [ ]:
purchase_data_by_month.shape

## Airings Sheet

In [ ]:
airings_data.info()

In [ ]:
# airings_data.groupby([pd.Grouper(key='Date/Time ET', freq='M'), 'Network'])[['Spend', 'Lift']].agg('sum')

## Preparing Lookup Data for Join

In [ ]:
lookup_data = lookup_data.drop('Exit Survey.1', axis=1)
# lookup_data = lookup_data.set_index('Exit Survey')
# lookup_data = lookup_data.rename_axis('Source')
lookup_data

In [ ]:
lookup_data.shape

## Joining Purchases by network and month to Lookup Data

In [ ]:
# joined_tbl = lookup_data[['Exit Survey', 'Airings']].merge(right=purchase_data_by_date, left_on='Exit Survey', right_on='Source', how='left')
# #joined_tbl.drop(labels='Exit Survey', axis=1, inplace=True)
# joined_tbl

In [ ]:
# purchase_grouped = purchase_data_by_month.join(lookup_data, how='right')
# purchase_grouped

In [ ]:
by_month_tbl = lookup_data.merge(right=purchase_data_by_month, left_on='Exit Survey', right_on='Source', how='left').set_index(['Exit Survey', 'date'])
by_month_tbl

In [ ]:
print(by_month_tbl.to_string())

In [ ]:
by_month_tbl.shape

## Spend and Lift by Network and Month

In [ ]:
# NEED TO drop Network as an index and make a column, join with purchase_grouped and keep the 
airings_spend_lift_grouped = airings_data.groupby(['Network', pd.Grouper(key='Date/Time ET', freq='M')])[['Spend', 'Lift']].agg('sum')
airings_spend_lift_grouped

In [ ]:
# airings_spend_lift_grouped.reset_index()

In [ ]:
by_month_tbl.reset_index().head()

In [ ]:
# purchase_grouped.reset_index()

In [ ]:
airings_spend_lift_grouped.reset_index().head()

## Joining Purchases/Lookup to Spend and Lift by Network and Month

In [ ]:
# month_and_network_grouped = purchase_grouped.reset_index().merge(right=airings_spend_lift_grouped.reset_index(), left_on=['Airings', 'date'], right_on=['Network', 'Date/Time ET'], how='left')
# month_and_network_grouped

new_tbl = by_month_tbl.reset_index().merge(right=airings_spend_lift_grouped.reset_index(), left_on=['Airings', 'date'], right_on=['Network', 'Date/Time ET'], how='left')
new_tbl.head()

In [ ]:
new_tbl.shape

In [ ]:
# month_and_network_grouped= month_and_network_grouped.set_index(['Source', 'date']).drop(labels=['Airings', 'Network', 'Date/Time ET'], axis=1)
# month_and_network_grouped

new_tbl = new_tbl.set_index(['Exit Survey', 'date']).drop(labels=['Airings', 'Network', 'Date/Time ET', 'Source'], axis=1)
new_tbl

In [ ]:
new_tbl.shape

In [ ]:
print(new_tbl.to_string())

In [ ]:
new_tbl['Conversion Rate'] = new_tbl['Purchases'] / new_tbl['Lift'] * 100
new_tbl['Cost Per Acquisition'] = new_tbl['Spend'] / new_tbl['Purchases']
new_tbl['Cost Per Visitor'] = new_tbl['Spend'] / new_tbl['Lift']
new_tbl['Percent of Purchases'] = new_tbl['Purchases'] / sum(new_tbl['Purchases'].fillna(0)) * 100
new_tbl['Percent of Spend'] = new_tbl['Spend'] / sum(new_tbl['Spend'].fillna(0)) * 100
new_tbl['Percent Pur > Percent Spend'] = new_tbl['Percent of Purchases'] > new_tbl['Percent of Spend']
new_tbl

In [ ]:
print(new_tbl.to_string())

## Done

In [ ]:
airings_data.query('Network == "FOOD"')

# What networks have purchases but no spend?

In [ ]:
airings_data.query('Spend == 0')['Network'].value_counts()

In [ ]:
airings_data.groupby('Network')[['Spend', 'Lift']].agg('sum')